In [1]:
ib.disconnect()

NameError: name 'ib' is not defined

In [2]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats.stats import pearsonr
import random
import json
from itertools import permutations
import numpy as np
import copy
import pickle
import time
import multiprocessing as mp
import math
import matplotlib.pyplot as plt
from IPython.display import display
from datetime import datetime
import datetime as dt
import ib_insync
print(ib_insync.__all__)

from ib_insync import *
util.startLoop()

#Connect to IB Gateway
ib = IB()
ib.connect('127.0.0.1', 4000, clientId=3,timeout=100)


['util', 'Event', 'SoftDollarTier', 'PriceIncrement', 'Execution', 'CommissionReport', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'ScannerSubscription', 'ScanDataList', 'FundamentalRatios', 'ExecutionFilter', 'PnL', 'PnLSingle', 'AccountValue', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'TickAttrib', 'TickAttribBidAsk', 'TickAttribLast', 'MktDepthData', 'DOMLevel', 'TradeLogEntry', 'FamilyCode', 'SmartComponent', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'ContFuture', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Bag', 'TagValue', 'ComboLeg', 'DeltaNeutralContract', 'ContractDetails', 'ContractDescrip

<IB connected to 127.0.0.1:4000 clientId=3>

In [ ]:
CurrentPortfolio = pd.DataFrame()
for element in ib.portfolio():
    symbol = str(element).split("symbol='")[1].split("'")[0]
#     primaryExchange = str(element).split("primaryExchange='")[1].split("'")[0]
    position =str(element).split("position=")[1].split(",")[0]
    unrealizedPNL = str(element).split("unrealizedPNL=")[1].split(",")[0]
    realizedPNL = str(element).split("realizedPNL=")[1].split(",")[0]
    marketValue = str(element).split("marketValue=")[1].split(",")[0]
    list_row = {"symbol":symbol, "position":position,"unrealizedPNL":unrealizedPNL,"realizedPNL":realizedPNL,"marketValue":marketValue}
    
    CurrentPortfolio = CurrentPortfolio.append(list_row, ignore_index=True)

In [ ]:
CurrentPortfolio

In [15]:
CurrentPortfolio = pd.DataFrame()
for element in ib.positions():
    symbol = str(element).split("symbol='")[1].split("'")[0]
    position =str(element).split("position=")[1].split(",")[0]
    list_row = {"symbol":symbol, "position":position}
    
    CurrentPortfolio = CurrentPortfolio.append(list_row, ignore_index=True)
CurrentPortfolio.index = CurrentPortfolio["symbol"]

In [4]:
contract = Stock("AAPL", 'SMART', 'USD')

In [19]:
CurrentPortfolio

,position,symbol
symbol,,
KEY,-260.0,KEY
KLAC,-3.0,KLAC
KEP,372.0,KEP
TOT,86.0,TOT
AXP,5.0,AXP
...,...,...
GGG,35.0,GGG
PE,-674.0,PE
PCTY,-11.0,PCTY


In [ ]:
CurrentPortfolio = pd.DataFrame()
for element in ib.portfolio():
    symbol = str(element).split("symbol='")[1].split("'")[0]
#     primaryExchange = str(element).split("primaryExchange='")[1].split("'")[0]
    position =str(element).split("position=")[1].split(",")[0]
    unrealizedPNL = str(element).split("unrealizedPNL=")[1].split(",")[0]
    realizedPNL = str(element).split("realizedPNL=")[1].split(",")[0]
    marketValue = str(element).split("marketValue=")[1].split(",")[0]
    list_row = {"symbol":symbol, "position":position,"unrealizedPNL":unrealizedPNL,"realizedPNL":realizedPNL,"marketValue":marketValue}
    
    CurrentPortfolio = CurrentPortfolio.append(list_row, ignore_index=True)

In [5]:
ib.portfolio()[0]

PortfolioItem(contract=Stock(conId=1715006, symbol='A', right='0', primaryExchange='NYSE', currency='USD', localSymbol='A', tradingClass='A'), position=8.0, marketPrice=89.973999, marketValue=719.79, averageCost=77.775, unrealizedPNL=97.59, realizedPNL=0.0, account='DU1870227')

In [6]:
ib.positions()[0]

Position(account='DU1870227', contract=Stock(conId=8809, symbol='KEY', exchange='NYSE', currency='USD', localSymbol='KEY', tradingClass='KEY'), position=-260.0, avgCost=10.39626615)

In [ ]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats.stats import pearsonr
import random
import json
from itertools import permutations
import numpy as np
import copy
import pickle
import time
import multiprocessing as mp
import math
import matplotlib.pyplot as plt
from IPython.display import display
from datetime import datetime
import datetime as dt
import ib_insync
print(ib_insync.__all__)

from ib_insync import *
util.startLoop()

#Connect to IB Gateway
ib = IB()
ib.connect('127.0.0.1', 4000, clientId=1,timeout=100)

def format_days(x):
    if len(x)<2:
        return("0"+x)
    else:
        return(x)
    
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

#inut file
target_position = pd.read_pickle("dailyorders.pickle")
target_position.index = pd.to_datetime(target_position.index)
target_position = target_position[target_position.index==target_position.index[-1]]


##insert the location of the exchange.
ExchangeData = pd.read_csv("exchange.csv")
ExchangeData.index= ExchangeData["Stock"]
ExchangeData.loc["AAPL"]["primaryExchange"]=="NASDAQ"
StrategyDay = str(target_position.index[0].year)+"-"+format_days(str(target_position.index[0].month))+"-"+format_days(str(target_position.index[0].day))


#Construct CurrentPortfolio
CurrentPortfolio = pd.DataFrame()
for element in ib.portfolio():
    symbol = str(element).split("symbol='")[1].split("'")[0]
#     primaryExchange = str(element).split("primaryExchange='")[1].split("'")[0]
    position =str(element).split("position=")[1].split(",")[0]
    unrealizedPNL = str(element).split("unrealizedPNL=")[1].split(",")[0]
    realizedPNL = str(element).split("realizedPNL=")[1].split(",")[0]
    marketValue = str(element).split("marketValue=")[1].split(",")[0]
    list_row = {"symbol":symbol, "position":position,"unrealizedPNL":unrealizedPNL,"realizedPNL":realizedPNL,"marketValue":marketValue}
    
    CurrentPortfolio = CurrentPortfolio.append(list_row, ignore_index=True)

for company in target_position.columns:
    if company not in CurrentPortfolio:
        symbol = company
        position = 0
        unrealizedPNL = 0 
        realizedPNL = 0
        marketValue = 0
        list_row = {"symbol":symbol, "position":position,"unrealizedPNL":unrealizedPNL,"realizedPNL":realizedPNL,"marketValue":marketValue}
        
        CurrentPortfolio = CurrentPortfolio.append(list_row, ignore_index=True)

CurrentPortfolio.index = CurrentPortfolio["symbol"]
PreviousPortfolio = CurrentPortfolio.copy()
excluse = {"UTX","RTN","S","ZAYO","GDI","SBGL","INXN","CY","MSG","LK"}



####

# My file destination.

#Currently not used
##existing_stocks = set(CurrentPortfolio["symbol"]).intersection(set(target_position))



###Execution
for element in target_position.columns:
    amount_portfolio = CurrentPortfolio[CurrentPortfolio.index==element]["position"] 
    new_amount_portfolio = target_position[target_position.index==StrategyDay][element]

    
    # ROUNDING
    amount_portfolio = int(round(amount_portfolio))
    new_amount_portfolio = int(round(new_amount_portfolio))
    
    # Buy / Sell / Hold
    if new_amount_portfolio>amount_portfolio:
        direction = "BUY"
        trade_amount = new_amount_portfolio - amount_portfolio
    if new_amount_portfolio<amount_portfolio:
        direction = "SELL"
        trade_amount = amount_portfolio - new_amount_portfolio
    if new_amount_portfolio==amount_portfolio:
        direction = "HOLD"
        trade_amount = 0
    
    if direction!="HOLD":
        if(stock not in excluse):
            try:
                contract = Stock(element, 'SMART', 'USD', primaryExchange=ExchangeData.loc[element]["primaryExchange"])
                order = MarketOrder(direction, trade_amount)
    #             trade = ib.placeOrder(contract, order)
#                 print(contract)
#                 print(order)
            except:
#                 print(element)
